# Temporal Features - RNN Test

In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten, GRU, SimpleRNN, BatchNormalization

import sqlalchemy
from sqlalchemy import create_engine, inspect

import math
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
from pprint import pprint

import os
import sys
import time
from datetime import datetime
from contextlib import redirect_stdout

%run functions.ipynb

In [2]:
# Time the run
start_time = time.time()

## Import datasets

In [3]:
# Import the data
engine = create_engine("sqlite:///voice.sqlite")

# View all of the classes
inspector = inspect(engine)
table_names = inspector.get_table_names()
table_names

['alexaval',
 'alexbval',
 'alexgval',
 'alexrval',
 'aval',
 'bval',
 'chroma1',
 'chroma10',
 'chroma11',
 'chroma12',
 'chroma2',
 'chroma3',
 'chroma4',
 'chroma5',
 'chroma6',
 'chroma7',
 'chroma8',
 'chroma9',
 'chromastd',
 'deltachroma1',
 'deltachroma10',
 'deltachroma11',
 'deltachroma12',
 'deltachroma2',
 'deltachroma3',
 'deltachroma4',
 'deltachroma5',
 'deltachroma6',
 'deltachroma7',
 'deltachroma8',
 'deltachroma9',
 'deltachromastd',
 'deltaenergy',
 'deltaenergyentropy',
 'deltamfcc1',
 'deltamfcc10',
 'deltamfcc11',
 'deltamfcc12',
 'deltamfcc13',
 'deltamfcc2',
 'deltamfcc3',
 'deltamfcc4',
 'deltamfcc5',
 'deltamfcc6',
 'deltamfcc7',
 'deltamfcc8',
 'deltamfcc9',
 'deltaspectralcentroid',
 'deltaspectralentropy',
 'deltaspectralflux',
 'deltaspectralrolloff',
 'deltaspectralspread',
 'deltazcr',
 'demographic',
 'diagnosis',
 'energy',
 'energyentropy',
 'gval',
 'habits',
 'mfcc1',
 'mfcc10',
 'mfcc11',
 'mfcc12',
 'mfcc13',
 'mfcc2',
 'mfcc3',
 'mfcc4',
 'mfcc5

In [4]:
# Tables not to import
no_import_tables = [
    'rval', 'gval', 'bval', 'aval',
    'demographic', 'habits',
    'alexrval', 'alexgval', 'alexbval', 'alexaval'
]

In [5]:
# Initialise a dictionary to hold dataframes
dataframes = dict()

# Loop through each table
for table in table_names:
    
    if table not in no_import_tables:
        # Dataframe name
        df_name = f'{table}_df'

        # Create dataframe
        dataframes[df_name] = pd.read_sql(
            f'SELECT * FROM {table}',
            engine
        )

## Preprocessing

### Define the target variable

In [6]:
# Isolate the diagnosis column
diagnosis_df = dataframes['diagnosis_df'].sort_values(by='id').reset_index(drop=True)
y = diagnosis_df['diagnosis'].copy()

# Encode the target variable, ignore subtype
y = y.apply(encode_binary)
y

0      1
1      0
2      1
3      1
4      1
      ..
199    0
200    1
201    0
202    0
203    1
Name: diagnosis, Length: 204, dtype: int64

### Recombine the feature variables

In [7]:
# Define subset of only temporal features
temporal_tables = list(dataframes.keys())
temporal_tables.remove('diagnosis_df')

# Initialise a dictionary to hold all features
all_feats = dict()

# Loop through each temporal feature
for table in sorted(temporal_tables):
    
    # Get the dataframe
    df = dataframes[table]
    
    # Get the feature array and voice IDs
    feat_array = df.values[:, 1:]
    voice_list = df.values[:, 0]
    
    # Initialise a feature dictionary
    feat_dict = dict()
    
    # Use a for-loop to populate the dictionary
    for idx, feat in enumerate(feat_array):
        feat_dict[voice_list[idx]] = feat
        # if len(feat) != 192:
            # print(len(feat))
    
    # Append to the all feature dictionary
    feat_name = table.split("_")[0]
    all_feats[feat_name] = feat_dict

# Convert to a dataframe
X = pd.DataFrame(all_feats).sort_index().reset_index(drop=True)
# X = pd.DataFrame(all_feats)
X.head()

,chroma10,chroma11,chroma12,chroma1,chroma2,chroma3,chroma4,chroma5,chroma6,chroma7,...,mfcc6,mfcc7,mfcc8,mfcc9,spectralcentroid,spectralentropy,spectralflux,spectralrolloff,spectralspread,zcr
0,"[8.160939658571425e-36, 8.160939658571425e-36,...","[6.259135778625184e-35, 6.259135778625184e-35,...","[6.521327966022236e-37, 6.521327966022236e-37,...","[1.981977565762179e-36, 1.981977565762179e-36,...","[1.3575144582002011e-36, 1.3575144582002011e-3...","[0.2, 0.2, 0.2, 0.0083557589856543, 0.00571074...","[4.104873703767106e-36, 4.104873703767106e-36,...","[2.155332585060541e-35, 2.155332585060541e-35,...","[5.559684454245396e-36, 5.559684454245396e-36,...","[1.3576749403691584e-33, 1.3576749403691584e-3...",...,"[1.1190399446969557e-07, 1.1190399446969557e-0...","[8.413179620042933e-08, 8.413179620042933e-08,...","[6.508014845220068e-08, 6.508014845220068e-08,...","[4.8439279586533656e-08, 4.8439279586533656e-0...","[0.005, 0.005, 0.005, 0.3465049654087774, 0.28...","[1.223008975714376e-10, 1.223008975714376e-10,...","[0.0, 0.0, 0.0, 0.9996124728099652, 0.00645907...","[0.0, 0.0, 0.0, 0.42, 0.325, 0.33, 0.325, 0.32...","[4.770690588753296e-09, 4.770690588753296e-09,...","[0.0, 0.0, 0.0, 0.0701754385964912, 0.20300751..."
1,"[7.013453879267504e-36, 7.013453879267504e-36,...","[0.0, 0.0, 0.0, 0.0166532558508933, 0.07869997...","[0.0, 0.0, 0.0, 0.0129537558999756, 0.00446014...","[0.0, 0.0, 0.0, 0.0030468925055301, 0.00093043...","[7.408793082486265e-37, 7.408793082486265e-37,...","[0.1999999999999999, 0.1999999999999999, 0.199...","[3.4822079082192796e-36, 3.4822079082192796e-3...","[3.341581545922013e-36, 3.341581545922013e-36,...","[0.0, 0.0, 0.0, 0.0251245180249349, 0.02791182...","[8.629140939612992e-35, 8.629140939612992e-35,...",...,"[5.570574598767376e-09, 5.570574598767376e-09,...","[2.3875995972436553e-09, 2.3875995972436553e-0...","[2.749349885292186e-10, 2.749349885292186e-10,...","[-1.6617109241744871e-09, -1.6617109241744871e...","[0.005, 0.005, 0.005, 0.2686992943604466, 0.24...","[1.6891478942563823e-09, 1.6891478942563823e-0...","[0.0, 0.0, 0.0, 1.015672065306574, 0.012061107...","[0.0, 0.0, 0.0, 0.305, 0.295, 0.295, 0.295, 0....","[3.0750895132596103e-09, 3.0750895132596103e-0...","[0.0, 0.0, 0.0, 0.0701754385964912, 0.18045112..."
2,"[0.0, 0.0, 0.0, 0.0254328104050869, 0.00097298...","[0.0, 0.0, 0.0, 0.0193505451629107, 0.00262743...","[0.0, 0.0, 0.0, 0.0083292562363225, 0.00160357...","[0.0, 0.0, 0.0, 0.0080703610497697, 0.01283563...","[0.0, 0.0, 0.0, 0.0070286755905986, 0.00614136...","[0.2, 0.2, 0.2, 0.0085329179281302, 0.00461212...","[0.0, 0.0, 0.0, 0.0052980532581862, 0.00109933...","[0.0, 0.0, 0.0, 0.00504925682982, 0.0052632757...","[0.0, 0.0, 0.0, 0.0059614712279919, 0.01369267...","[0.0, 0.0, 0.0, 0.0128160916733445, 0.01380721...",...,"[0.0, 0.0, 0.0, -0.2599778999084198, -0.290567...","[0.0, 0.0, 0.0, 0.1582810230304283, 0.22599124...","[0.0, 0.0, 0.0, -0.1396072459697239, -0.235683...","[0.0, 0.0, 0.0, -0.2101056367575636, -0.212658...","[0.0049999999999999, 0.0049999999999999, 0.004...","[7.084185505857156e-10, 7.084185505857156e-10,...","[0.0, 0.0, 0.0, 1.0014504084715852, 0.01066764...","[0.0, 0.0, 0.0, 0.33, 0.32, 0.32, 0.32, 0.32, ...","[8.88178419700125e-19, 8.88178419700125e-19, 8...","[0.0, 0.0, 0.0, 0.0651629072681704, 0.17293233..."
3,"[2.702574583604019e-36, 2.702574583604019e-36,...","[4.776934641543469e-37, 4.776934641543469e-37,...","[0.0, 0.0, 0.0, 0.0090083795612209, 0.00081537...","[1.1698336326834081e-34, 1.1698336326834081e-3...","[7.2024558647616085e-37, 7.2024558647616085e-3...","[0.2, 0.2, 0.2, 0.0356911864166217, 0.00370847...","[1.3418390923476396e-36, 1.3418390923476396e-3...","[1.2876499240617028e-36, 1.2876499240617028e-3...","[0.0, 0.0, 0.0, 0.0065263846329425, 0.00703412...","[7.234653246625582e-35, 7.234653246625582e-35,...",...,"[-9.094534884871259e-08, -9.094534884871259e-0...","[-5.831674649803382e-07, -5.831674649803382e-0...","[-8.326572895673644e-07, -8.3265728956

In [8]:
# Split the preprocessed data to training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [9]:
# Check shapes
len(X_train), len(X_test), len(X.columns), len(X['chroma10'][0])

(153, 51, 68, 192)

In [10]:
# Flatten the feature arrays
X_train_flat = np.array(
    [X[feature][i] for feature in X.keys() for i in range(len(X_train))])

X_test_flat = np.array(
    [X[feature][i] for feature in X.keys() for i in range(len(X_test))])

In [11]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()

In [12]:
# Use fit_transform on X_train
X_train_scaled = scaler.fit_transform(
    X_train_flat.reshape(
        len(X_train) * len(X.columns) * len(X['chroma10'][0]),
        -1
    )).reshape(
        len(X_train),
        len(X.columns),
        len(X['chroma10'][0])
    )

# Use transform on X_Test
X_test_scaled = scaler.transform(
    X_test_flat.reshape(
        len(X_test) * len(X.columns) * len(X['chroma10'][0]),
        -1
    )).reshape(
        len(X_test),
        len(X.columns),
        len(X['chroma10'][0])
    )

In [13]:
# Define the RNN model
rnn = Sequential()

# Define the input shape
input_shape = (len(X.columns), len(X['chroma10'][0]))


# rnn.add(Conv1D(
#     filters = 32,
#     kernel_size = 3,
#     activation = 'relu',
#     input_shape = input_shape
# ))
# rnn.add(MaxPooling1D(pool_size=2))
# rnn.add(Flatten())

# Add the first layer
rnn.add(SimpleRNN(
    units = 192,
    input_shape = input_shape,
    return_sequences = True,
    activation = 'relu'
))

rnn.add(GRU(256, return_sequences=True))
rnn.add(BatchNormalization())
rnn.add(GRU(256, return_sequences=False))
rnn.add(BatchNormalization())
# Add the second layer
# rnn.add(LSTM(
#     units = 192,
#     return_sequences = True,
#     activation = 'tanh'
# ))

# # Add the third layer
# rnn.add(LSTM(
#     units = 192,
#     return_sequences = False,
#     activation = 'tanh'
# ))

# Add the output layer
rnn.add(Dense(units=1, activation='sigmoid'))

# Build the model, required for Bidirectional
# rnn.build(input_shape=(None, input_shape[0], input_shape[1]))

# Display the summary
rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 68, 192)           73920     
                                                                 
 gru (GRU)                   (None, 68, 256)           345600    
                                                                 
 batch_normalization (Batch  (None, 68, 256)           1024      
 Normalization)                                                  
                                                                 
 gru_1 (GRU)                 (None, 256)               394752    
                                                                 
 batch_normalization_1 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dense (Dense)               (None, 1)                 2

In [14]:
# Compile the model
rnn.compile(
    loss = "binary_crossentropy",
    optimizer = "adam",
    metrics = ["accuracy"]
)

In [15]:
# Train the model
fit_model = rnn.fit(
    X_train_scaled,
    y_train,
    shuffle = True,
    epochs = 20,
    verbose = 1
)

Epoch 1/20
5/5 [==============================] - 2s 147ms/step - loss: 0.7785 - accuracy: 0.4902
Epoch 2/20
5/5 [==============================] - 1s 165ms/step - loss: 0.8657 - accuracy: 0.5556
Epoch 3/20
5/5 [==============================] - 1s 176ms/step - loss: 0.7570 - accuracy: 0.6013
Epoch 4/20
5/5 [==============================] - 1s 151ms/step - loss: 0.6752 - accuracy: 0.6993
Epoch 5/20
5/5 [==============================] - 1s 175ms/step - loss: 0.6539 - accuracy: 0.7059
Epoch 6/20
5/5 [==============================] - 1s 171ms/step - loss: 0.6365 - accuracy: 0.7190
Epoch 7/20
5/5 [==============================] - 1s 181ms/step - loss: 0.6587 - accuracy: 0.7124
Epoch 8/20
5/5 [==============================] - 1s 159ms/step - loss: 0.6220 - accuracy: 0.7059
Epoch 9/20
5/5 [==============================] - 1s 175ms/step - loss: 0.6003 - accuracy: 0.7190
Epoch 10/20
5/5 [==============================] - 1s 196ms/step - loss: 0.5892 - accuracy: 0.7255
Epoch 11/20
5/5 [==

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = rnn.evaluate(
    X_test_scaled,
    y_test,
    verbose = 2
)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2/2 - 0s - loss: 0.6830 - accuracy: 0.7647 - 343ms/epoch - 172ms/step
Loss: 0.6830335259437561, Accuracy: 0.7647058963775635


## Save Results to Performance Tracker

In [17]:
# Create dictionary to save the results
results_dict = dict()

current_time = datetime.now()
results_dict['timestamp'] = current_time
results_dict['runtime'] = time.time() - start_time
results_dict['model_loss'] = model_loss
results_dict['model_accuracy'] = model_accuracy

In [18]:
# Change message
change_message = input("Changes from previous iteration: ")

# Append to results_dict
results_dict['change_message'] = change_message

Changes from previous iteration:  added another batch normalisation after GRU


In [19]:
# Display the dictionary
results_dict

{'timestamp': datetime.datetime(2024, 1, 8, 19, 31, 12, 55469),
 'runtime': 21.00733518600464,
 'model_loss': 0.6830335259437561,
 'model_accuracy': 0.7647058963775635,
 'change_message': 'added another batch normalisation after GRU'}

In [20]:
# Model summary folder path
summary_path = '../resources/tracker/rnn_summary/'
num_files = len(os.listdir(summary_path))

# Export summary to text file
with open(f'{summary_path}rnn_summary_{num_files}.txt', 'w') as f:
    with redirect_stdout(f):
        rnn.summary()
        pprint(results_dict)

In [21]:
# Convert the dictionary to a dataframe
results_df = pd.DataFrame(results_dict, index=[0])
results_df.head()

,timestamp,runtime,model_loss,model_accuracy,change_message
0,2024-01-08 19:31:12.055469,21.007335,0.683034,0.764706,added another batch normalisation after GRU


In [22]:
# Performance tracker
tracker_path = "../resources/tracker/rnn_performance_tracker.csv"

# Model percentage
model_pct = round(model_accuracy, 3)

# Check if the CSV exists
if os.path.exists(tracker_path):
    
    # Read the existing CSV
    tracker_df = pd.read_csv(tracker_path)
    
    # Append the new row of data
    updated_df = pd.concat([tracker_df, results_df], ignore_index=True)
    
    # Update the CSV file
    updated_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    rnn.save(f'../models/rnn/run_{len(tracker_df)}_{model_pct}.h5', save_format='h5')

else:    
    # Export to CSV
    results_df.to_csv(tracker_path, index=False)
    
    # Export model to HDF5 file
    rnn.save(f'../models/rnn/run_0_{model_pct}.h5', save_format='h5')

/Users/alyssahondrade/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Understand the Predictions

In [23]:
# Check the prediction's output probabilities
predicted_prob = rnn.predict(X_test_scaled)
clean_prob = np.round(predicted_prob, 2)

# Round to the nearest integer and flatten
clean_predicted = np.round(predicted_prob).astype(int).flatten()

# Convert to a dataframe for readability
output_prob = pd.DataFrame({
    'Actual': y_test,
    'Predicted': clean_predicted,
    'Probability': clean_prob.flatten()
})
output_prob.head(10)

2/2 [==============================] - 0s 40ms/step


,Actual,Predicted,Probability
149,1,1,0.51
130,1,1,0.51
191,0,1,0.51
121,1,1,0.51
140,1,1,0.51
138,1,1,0.51
95,0,1,0.51
177,0,1,0.51
124,1,1,0.51
8,1,1,0.51


In [24]:
# Display incorrect indices
incorrect_idx = output_prob.loc[output_prob['Actual'] != output_prob['Predicted']].index
incorrect_idx

Index([191, 95, 177, 46, 102, 172, 153, 87, 201, 80, 110, 86], dtype='int64')

In [25]:
# Display incorrect predictions as a complete dataframe
dataframes['diagnosis_df'].iloc[incorrect_idx]

,id,diagnosis,subtype,vhi_score,rsi_score,reflux_indicated,vhi_zscore,vhi_impact
191,voice067,healthy,no subtype,7,13,1,-0.12,0
95,voice144,hypokinetic dysphonia,no subtype,59,21,1,3.36,4
177,voice010,hypokinetic dysphonia,no subtype,9,12,0,0.02,1
46,voice123,healthy,no subtype,0,3,0,-0.58,0
102,voice133,reflux laryngitis,no subtype,44,8,0,2.35,3
172,voice032,healthy,no subtype,16,18,1,0.48,1
153,voice147,hyperkinetic dysphonia,adduction deficit,47,9,0,2.56,3
87,voice031,reflux laryngitis,no subtype,16,21,1,0.48,1
201,voice165,hypokinetic dysphonia,glottic insufficiency,97,16,1,5.90,4
80,voice064,hyperkinetic dysphonia,cordite,31,0,0,1.49,2
